In [1]:
%load_ext lab_black
%load_ext autotime
import numpy as np
import pandas as pd

time: 374 ms (started: 2023-06-26 22:02:24 -07:00)


A dataset [mapping (median) psychological traits to professions](https://www.pnas.org/content/116/52/26459) ([github repo](https://github.com/behavioral-ds/VocationMap)). Spotted in [a tweet by Arseny Khakhalin](https://twitter.com/ampanmdagaba/status/1671943894232170496). A related dataset is the [O\*NET database](https://github.com/jlmelville/drnb/blob/c9d61299e6fcb88c0d8c10a3863d0562d5d33469/notebooks/data-pipeline/onet.ipynb).

In [2]:
data = pd.read_csv(
    "https://raw.githubusercontent.com/behavioral-ds/VocationMap/master/data/profession-profiles.csv",
    header=0,
    index_col=0,
)
data

,Openness,Conscientousness,Extraversion,Agreeableness,Emotional_Range,Conversation,Openness to Change,Hedonism,Self-enhancement,Self-transcendence,n
Profession,,,,,,,,,,,
.Net Architect,0.683985,0.342936,0.163002,0.017733,0.689112,0.041652,0.309403,0.126749,0.270632,0.199605,79
.NET Developer,0.714061,0.263780,0.124042,0.039167,0.599707,0.051834,0.317143,0.180365,0.303151,0.209806,81
.NET Programmer,0.678342,0.317171,0.193691,0.060354,0.635676,0.083598,0.311900,0.148964,0.293216,0.245225,64
.NET Web Developer,0.713994,0.275925,0.229268,0.042219,0.607029,0.062180,0.334345,0.158734,0.279614,0.212778,77
Academic Coordinator,0.621694,0.688403,0.721175,0.561432,0.646653,0.355991,0.458598,0.120948,0.373753,0.377974,76
...,...,...,...,...,...,...,...,...,...,...,...
Wine Maker,0.777015,0.675275,0.808460,0.373987,0.670210,0.213829,0.349691,0.168718,0.278539,0.230143,79
Wireless Engineer,0.679632,0.461463,0.377782,0.093101,0.603731,0.106208,0.375550,0.167429,0.381982,0.198991,63
World Designer,0.829150,0.523642,0.491623,0.323737,0.512637,0.140082,0.605152,0.232899,0.414527,0.328630,77


time: 580 ms (started: 2023-06-26 22:02:25 -07:00)


`n` is the number of people with that profession who contributed to these average values, so we won't include that.

In [3]:
data = data.iloc[:, :-1]
data

,Openness,Conscientousness,Extraversion,Agreeableness,Emotional_Range,Conversation,Openness to Change,Hedonism,Self-enhancement,Self-transcendence
Profession,,,,,,,,,,
.Net Architect,0.683985,0.342936,0.163002,0.017733,0.689112,0.041652,0.309403,0.126749,0.270632,0.199605
.NET Developer,0.714061,0.263780,0.124042,0.039167,0.599707,0.051834,0.317143,0.180365,0.303151,0.209806
.NET Programmer,0.678342,0.317171,0.193691,0.060354,0.635676,0.083598,0.311900,0.148964,0.293216,0.245225
.NET Web Developer,0.713994,0.275925,0.229268,0.042219,0.607029,0.062180,0.334345,0.158734,0.279614,0.212778
Academic Coordinator,0.621694,0.688403,0.721175,0.561432,0.646653,0.355991,0.458598,0.120948,0.373753,0.377974
...,...,...,...,...,...,...,...,...,...,...
Wine Maker,0.777015,0.675275,0.808460,0.373987,0.670210,0.213829,0.349691,0.168718,0.278539,0.230143
Wireless Engineer,0.679632,0.461463,0.377782,0.093101,0.603731,0.106208,0.375550,0.167429,0.381982,0.198991
World Designer,0.829150,0.523642,0.491623,0.323737,0.512637,0.140082,0.605152,0.232899,0.414527,0.328630


time: 15.2 ms (started: 2023-06-26 22:02:25 -07:00)


The cluster assignments shown on the vocation map itself are the medoids found from running the R function for performing the Partition Around Medoids (PAM) using the `cluster::pam` function. See: <https://github.com/behavioral-ds/VocationMap/blob/8d8455560e5e1de667399333afb733c7bb3c1209/scripts/vocation-map.R#L29>. You can also find the colors in there too, although I don't quite understand how they were picked: from the "Dark2" and "Paired" [ColorBrewer palettes](https://colorbrewer2.org/), it seems.

Rather than make you load the data into R, I went to the [interactive version's HTML](https://raw.githubusercontent.com/behavioral-ds/VocationMap/master/plots/vocation-map-interactive.html), copied out the plotly data and then pasted it into a JavaScript console. In the `x.data` array, the `name` attribute is the cluster name, and the `text` attribute is an array of the names of professions, although with the text ` (<x> users)` appended so you'll want to strip that off too.

For what it's worth, here is the JavaScript I used to scrape this stuff out (I assigned the data to a variable `vocation_map`): 

```javascript
// array of cluster names
vocation_map.x.data.map(n => n.name)

// array of array of professions
JSON.stringify(vocation_map.x.data.slice(0, vocation_map.x.data.length - 1).map(professions => {
    return professions.text.map(profession => {
        return profession.replace(/ \(\d+ users\)$/, '');
    });
}))

// converting to hex colors
function rgbToHex(rgb){
    let regex = /^rgba?\((\d+),\s*(\d+),\s*(\d+)(?:,\s*(\d+(?:\.\d+)?))?\)$/;
    let matches = rgb.match(regex);
    if(matches){
        let r = parseInt(matches[1]);
        let g = parseInt(matches[2]);
        let b = parseInt(matches[3]);

        return "#" + ((1 << 24) + (r << 16) + (g << 8) + b).toString(16).slice(1).toUpperCase();
    } else {
        return null; // Invalid RGB or RGBA color
    }
}

JSON.stringify(clusters.x.data.slice(0, clusters.x.data.length - 1).map(x => rgbToHex(x.marker.color)))
```

In [4]:
clusters = {
    "Business Analyst": [
        "Advertising Director",
        "Aerospace Engineer",
        "Analyst",
        "Analyst Sales",
        "Architect Manager",
        "Asset Manager",
        "Aviation Consultant",
        "Aviation Manager",
        "Bank Analyst",
        "Banking Analyst",
        "Brewing Director",
        "Business Analyst",
        "Business Management Analyst",
        "Conference Organizer",
        "Construction Engineer",
        "Consultant",
        "Customer Engineer",
        "Data Officer",
        "Design Engineer",
        "Digital Strategist",
        "Director of Product Development",
        "Director of Technology",
        "Engineer",
        "Engineering Director",
        "Equity Analyst",
        "Finance Analyst",
        "Financial Analyst",
        "Financial Engineer",
        "Futures Trader",
        "Hedge Fund Manager",
        "Internet Designer",
        "Investment Analyst",
        "Investment Banker",
        "Investment Fund Manager",
        "Investment Manager",
        "Investment Officer",
        "Investments Manager",
        "Investor",
        "Land Developer",
        "Management Analyst",
        "Market Analyst",
        "Market Research Analyst",
        "Market Researcher",
        "Money Manager",
        "Network Consultant",
        "Network Specialist",
        "Operations Analyst",
        "Portfolio Manager",
        "Principal Consultant",
        "Product Marketing Manager",
        "Product Trainer",
        "Project Leader",
        "Risk Manager",
        "Sales Engineer",
        "Staff Analyst",
        "Stock Analyst",
        "Stock Trader",
        "Strategist",
        "Supply Chain Specialist",
        "Systems Manager",
        "Technical Account Manager",
        "Technical Instructor",
        "Technical Recruiter",
        "Testing Director",
        "Trade Analyst",
        "Trader",
        "Transportation Engineer",
        "Wrecker",
    ],
    "Certified Financial Planner (CFP)": [
        "Advisor",
        "Benefits Consultant",
        "Broker",
        "Bureau Chief",
        "Bursar",
        "Buyer",
        "Buying Agent",
        "Car Dealer",
        "Certified Financial Planner (CFP)",
        "Certified Public Accountant",
        "Commercial Lender",
        "Compliance Consultant",
        "Contractor",
        "Dealer",
        "Ecommerce Marketing Manager",
        "Electrical Contractor",
        "Examiner",
        "Internet Consultant",
        "Internet Marketing Consultant",
        "Internet Marketing Specialist",
        "Internet Marketing Strategist",
        "Internet Retailer",
        "Landlord",
        "Liquidator",
        "Loan Originator",
        "Manufacturer",
        "Mortgage Banker",
        "Mortgage Broker",
        "Mortgage Consultant",
        "Mortgage Originator",
        "Mortgage Specialist",
        "Online Marketing Strategist",
        "Personal Financial Planner",
        "Public Accountant",
        "Real Estate Appraiser",
        "Roofing Contractor",
        "Search Consultant",
        "Search Marketing Specialist",
        "Search Strategist",
        "Systems Integrator",
        "Tax Accountant",
        "Tax Agent",
        "Tax Expert",
        "Tax Professional",
        "Wealth Management Advisor",
        "Web Marketing Strategist",
    ],
    "City Alderman": [
        "Adjuster",
        "Alderman",
        "Assessor",
        "Association Executive",
        "Auditor",
        "Benefits Specialist",
        "Book Publisher",
        "Bookkeeper",
        "Business Dean",
        "Business Representative",
        "Chairman",
        "Chancellor",
        "Chief Deputy",
        "Chief of Staff",
        "City Alderman",
        "City Council Member",
        "City Councilman",
        "Coffee Farmer",
        "College Dean",
        "Communications Director",
        "Community Center Director",
        "Comptroller",
        "Council Member",
        "Councilman",
        "Councilor",
        "Department Director",
        "Enforcement Officer",
        "Executive Vice President",
        "Farmer",
        "Financial Agent",
        "Fire Chief",
        "Food Broker",
        "Foreign Service Officer",
        "Fundraising Specialist",
        "Home Designer",
        "Information Officer",
        "Inspector General",
        "Insurance Consultant",
        "Legislative Assistant",
        "Lender",
        "Loan Officer",
        "Media Relations Director",
        "Merchant",
        "Mortgage Loan Officer",
        "Mystery Shopper",
        "Newspaper Publisher",
        "Online Merchant",
        "Online Retailer",
        "President",
        "President of the United States",
        "Provost",
        "Public Affairs Director",
        "Public Affairs Specialist",
        "Public Information Director",
        "Public Information Officer",
        "Public Safety Director",
        "Publisher",
        "Reserve Officer",
        "School Commissioner",
        "Secretary",
        "Selectman",
        "Tax Collector",
        "Tax Preparer",
        "Treasurer",
        "University Dean",
        "Vice President",
    ],
    "Communications Specialist": [
        "Bureau Director",
        "Campaign Director",
        "Campaign Manager",
        "Cattleman",
        "College Director",
        "Communications Consultant",
        "Communications Coordinator",
        "Communications Specialist",
        "Compliance Officer",
        "Content Specialist",
        "Content Strategist",
        "Department Manager",
        "Development Associate",
        "Director of Communications",
        "Director of Finance",
        "Director of Public Affairs",
        "Division Chief",
        "Employment Advisor",
        "Employment Specialist",
        "Executive Director",
        "Farm Specialist",
        "Field Director",
        "Finance Associate",
        "Finance Director",
        "Financial Adviser",
        "Financial Director",
        "Foreign Diplomat",
        "General Agent",
        "Grant Writer",
        "Healthcare Consultant",
        "Healthcare Manager",
        "reader",
        "Information Specialist",
        "Insurance Writer",
        "Investigator",
        "Journeyman",
        "Magazine Publisher",
        "Managing Director",
        "Network Director",
        "Newspaper Manager",
        "Operations Director",
        "Policy Advisor",
        "Postmaster",
        "Program Analyst",
        "Project Development Director",
        "Publication Director",
        "Rancher",
        "Research Specialist",
        "Safety Coordinator",
        "Senior Consultant",
        "Service Director",
        "Sommelier",
        "Special Agent",
        "Strategic Planner",
        "Team Leader",
        "Trade Specialist",
        "Traffic Director",
        "Trustee",
        "Underwriter",
        "Union Organizer",
        "University Administrator",
        "Web Content Manager",
        "Web Manager",
        "Web Producer",
        "Wine Maker",
    ],
    "Concert Manager": [
        "Artist Agent",
        "Artist Manager",
        "Artist's Manager",
        "Assistant Manager",
        "Band Manager",
        "Booking Agent",
        "Boxing Promoter",
        "Brand Manager",
        "Buyer Assistant",
        "Celebrity Manager",
        "Commercial Director",
        "Community Manager",
        "Concert Manager",
        "Concert Promoter",
        "Crew Leader",
        "Event Manager",
        "Event Organizer",
        "Event Promoter",
        "Events Director",
        "Events Manager",
        "Fight Manager",
        "Fitness Manager",
        "Graphic Engineer",
        "Gym Manager",
        "House Manager",
        "Logistics Coordinator",
        "Merchandise Manager",
        "Music Agent",
        "Music Promoter",
        "Operations Manager",
        "Production Coordinator",
        "Production Manager",
        "Project Manager",
        "Promoter",
        "Promotion Manager",
        "Promotions Director",
        "Promotions Manager",
        "Purser",
        "Radio Station Manager",
        "Records Manager",
        "Shipper",
        "Social Media Manager",
        "Station Manager",
        "Talent Buyer",
        "Theatre Manager",
        "Venue Manager",
    ],
    "Director of Education": [
        "Academic Dean",
        "Administrative Officer",
        "Architectural Designer",
        "Art Dealer",
        "Arts Administrator",
        "Associate Dean",
        "Clinic Director",
        "Coordinator",
        "Curriculum Developer",
        "Curriculum Director",
        "Curriculum Writer",
        "Department Chair",
        "Director of Education",
        "Director of Health Education",
        "Director of Social Services",
        "Director of Sustainability",
        "Division Chair",
        "Division Director",
        "Education Coordinator",
        "Education Dean",
        "Education Director",
        "Education Specialist",
        "Educational Director",
        "Emergency Responder",
        "Engagement Director",
        "Faculty Dean",
        "Faculty Member",
        "Graduate School Dean",
        "Grant Officer",
        "Health Administrator",
        "Health Commissioner",
        "Health Director",
        "Health Manager",
        "House Designer",
        "Landscape Designer",
        "Learning and Development Consultant",
        "Learning Consultant",
        "Learning Coordinator",
        "Learning Officer",
        "Learning Specialist",
        "Library Consultant",
        "Library Director",
        "Mediator",
        "Network Coordinator",
        "Nonprofit Director",
        "Organizational Development Consultant",
        "Outreach Coordinator",
        "Police Commissioner",
        "Program Officer",
        "Project Director",
        "Project Executive",
        "Public Health Director",
        "Regional Director",
        "Residential Designer",
        "Safety Director",
        "Social Services Director",
        "Sustainability Coach",
        "Sustainability Coordinator",
        "Sustainability Director",
        "Sustainability Manager",
        "Sustainability Officer",
        "Teacher Education Director",
        "Welfare Director",
    ],
    "District Manager": [
        "Account Supervisor",
        "Administrative Assistant",
        "Admissions Counselor",
        "Associate Vice President",
        "Athlete Manager",
        "Athletic Coordinator",
        "Athletic Director",
        "Athletics Director",
        "Business Coordinator",
        "Campus Coordinator",
        "Casino Manager",
        "Catering Director",
        "College Recruiter",
        "Corporate Event Planner",
        "Corporate Planner",
        "Director",
        "Director of Personnel",
        "Director of Sales and Marketing",
        "Director of Sales Marketing",
        "District Manager",
        "Executive Chef",
        "Factor",
        "Fitness Coordinator",
        "Food General Manager",
        "Gauger",
        "General Manager (GM)",
        "High School Assistant Principal",
        "Hospitality Manager",
        "Hotel Director",
        "Hotel General Manager",
        "Marketing Planner",
        "Meeting Manager",
        "Meeting Planner",
        "Merchandising Manager",
        "National Sales Manager",
        "Operations Assistant",
        "Party Planner",
        "Personnel Director",
        "Quality Control",
        "Real Estate Agent",
        "Recruiting Coordinator",
        "Regional Sales Manager",
        "Resort Manager",
        "Restaurant General Manager",
        "Restaurant Manager",
        "Restaurant Owner",
        "Restauranteur",
        "Sales Coordinator",
        "Sales Marketing Director",
        "Skills Trainer",
        "Special Events Director",
        "Sports Agent",
        "Trainer",
        "University Relations Director",
        "Vice President of Operations",
    ],
    "Fashion Marketer": [
        "Artist Representative",
        "Booking Manager",
        "Creative Director",
        "Director of Entertainment",
        "E-Business Manager",
        "Entertainment Director",
        "Entertainment Manager",
        "Event Coordinator",
        "Event Planner",
        "Event Specialist",
        "Fashion Buyer",
        "Fashion Coordinator",
        "Fashion Marketer",
        "Image Consultant",
        "Lifestyle Director",
        "Marketing Assistant",
        "Merchandiser",
        "Music Executive",
        "Promotions Coordinator",
        "Promotions Producer",
        "Public Relations Coordinator",
        "Salon Manager",
        "Special Events Coordinator",
        "Talent Agent",
        "Talent Manager",
        "Theater Manager",
        "Theatrical Agent",
    ],
    "Group Manager": [
        "Account Manager",
        "Accountant Manager",
        "Accounts Manager",
        "Advertising Account Manager",
        "Advertising Manager",
        "Agent",
        "Area Sales Manager",
        "Bank Manager",
        "Banking Manager",
        "Booker",
        "Branch Manager",
        "Business Administrator",
        "Business Agent",
        "Business Development Analyst",
        "Business Development Director",
        "Business Development Manager",
        "Business Manager",
        "Camp Director",
        "Camp Manager",
        "Cash Manager",
        "Category Manager",
        "Catering Manager",
        "Client Services Director",
        "Club Manager",
        "Communication Manager",
        "Conference Manager",
        "Construction Manager",
        "Construction Project Manager",
        "Cook Manager",
        "Development Coordinator",
        "Development Officer",
        "Digital Marketing Manager",
        "Director of Development",
        "Director of Marketing",
        "Distribution Manager",
        "Division Manager",
        "Entertainment Agent",
        "Export Manager",
        "Facilities Director",
        "Factory Manager",
        "Fitness Director",
        "Food Manager",
        "General Manager",
        "Golf Club Manager",
        "Grant Manager",
        "Group Manager",
        "Head of Marketing",
        "Hotel Manager",
        "Internal Auditor",
        "Key Account Manager",
        "Location Manager",
        "Logistics Manager",
        "Marketing Communications Manager",
        "Marketing Coordinator",
        "Marketing Director",
        "Marketing Manager",
        "Marketing Representative",
        "Marketing Specialist",
        "Media Buyer",
        "Media Director",
        "Media Planner",
        "Media Promoter",
        "Music Publisher",
        "Operations Coordinator",
        "Operations Supervisor",
        "Performance Consultant",
        "Pharmacy Manager",
        "Production Supervisor",
        "Program Director",
        "Property Developer",
        "Purchasing Manager",
        "Recruitment Consultant",
        "Recruitment Director",
        "Recruitment Manager",
        "Regional Manager",
        "Relationship Manager",
        "Resident Manager",
        "Revenue Manager",
        "Sales Account Manager",
        "Sales Director",
        "School Business Manager",
        "Senior Instructor",
        "Senior Project Manager",
        "Social Media Director",
        "Social Media Specialist",
        "Sports Marketer",
        "Store Manager",
        "Team Coordinator",
        "Territory Manager",
        "Training Assistant",
        "Training Manager",
        "Zone Manager",
    ],
    "Health Care Manager": [
        "Account Specialist",
        "Art Coordinator",
        "Assembly Member",
        "Author's Agent",
        "Campaigner",
        "Case Manager",
        "Caseworker",
        "Clinical Director",
        "Clinical Manager",
        "Clinical Researcher",
        "Coroner",
        "Director of Housing",
        "Director of Quality",
        "Health Care Manager",
        "Health Education Director",
        "Health Services Director",
        "Hospital Director",
        "Hospital Manager",
        "Housing Director",
        "Housing Manager",
        "Housing Officer",
        "Landlady",
        "Learning Manager",
        "Liaison Officer",
        "Literary Agent",
        "Medical Director",
        "Medical Officer",
        "Nurse Manager",
        "Nursing Director",
        "Paramedic",
        "Practice Manager",
        "Publicity Director",
        "Publicity Manager",
        "Quality Manager",
        "Rehabilitation Director",
        "Stockman",
        "Training Officer",
        "Trust Officer",
        "Vice Chancellor",
        "Vice Principal",
        "Welfare Manager",
        "Welfare Officer",
    ],
    "Information Engineer": [
        ".Net Architect",
        ".NET Programmer",
        "Actuary",
        "Analyst Programmer",
        "Application Architect",
        "Application Developer",
        "Applications Developer",
        "Applications Engineer",
        "Automation Engineer",
        "Beta Tester",
        "Business Programmer",
        "Computer Analyst",
        "Computer Architect",
        "Computer Consultant",
        "Computer Instructor",
        "Computer Science Professor",
        "Computer Technician",
        "Computer Technologist",
        "Cyber Security Engineer",
        "Data Architect",
        "Data Manager",
        "Data Miner",
        "Database Consultant",
        "Database Engineer",
        "Database Manager",
        "Developer",
        "Enterprise Architect",
        "Front End Web Developer",
        "Information Analyst",
        "Information Engineer",
        "Information Security Analyst",
        "Information Security Engineer",
        "Infrastructure Architect",
        "Internet Architect",
        "Internet Developer",
        "Internet Programmer",
        "IT Specialist",
        "Java Software Developer",
        "Java Software Engineer",
        "Job Developer",
        "Knowledge Engineer",
        "Land Surveyor",
        "Linux Administrator",
        "Multimedia Developer",
        "Network Administrator",
        "Network Architect",
        "Network Developer",
        "Network Security Engineer",
        "Networking Engineer",
        "Oracle Developer",
        "PHP Developer",
        "PHP Web Developer",
        "Project Developer",
        "Quality Engineer",
        "Scrum Master",
        "Security Architect",
        "Security Consultant",
        "SharePoint Architect",
        "Sharepoint Developer",
        "Software Analyst",
        "Software Design Engineer",
        "Software Development Engineer",
        "Software Project Manager",
        "Solution Architect",
        "Solutions Architect",
        "Sports Statistician",
        "SQL Server Developer",
        "Statistician",
        "Storage Engineer",
        "System Administrator",
        "System Architect",
        "System Engineer",
        "System Software Developer",
        "Systems Architect",
        "Systems Specialist",
        "Technical Analyst",
        "Technical Consultant",
        "Technical Trainer",
        "Technology Manager",
        "Training Developer",
        "User Interface Designer",
        "Web Administrator",
        "Web Application Developer",
        "Web Applications Developer",
        "Web Architect",
        "Web Site Developer",
        "Wireless Engineer",
    ],
    "Management Scientist": [
        "Academic Director",
        "Agriculture Manager",
        "Architect",
        "Biostatistician",
        "Building Architect",
        "City Designer",
        "Collections Manager",
        "Constructor",
        "Dairy Farmer",
        "Data Scientist",
        "Design Architect",
        "Design Director",
        "Director of Research",
        "Emergency Management Director",
        "Environmental Analyst",
        "Environmental Designer",
        "Environmental Planner",
        "Environmental Scientist",
        "Environmental Specialist",
        "Fruit Grower",
        "Horticulturist",
        "Informaticist",
        "Information Architect",
        "Information Scientist",
        "Knowledge Manager",
        "Lab Manager",
        "Laboratory Director",
        "Land Agent",
        "Landscape Architect",
        "Management Scientist",
        "Mapper",
        "Product Scientist",
        "Project Architect",
        "Research and Development Director",
        "Research Associate",
        "Research Associate Professor",
        "Research Coordinator",
        "Research Development Director",
        "Research Development Manager",
        "Research Director",
        "Research Manager",
        "Researcher",
        "Risk Analyst",
        "Scientist",
        "Senior Research Associate",
        "Senior Research Scientist",
        "Senior Scientist",
        "Space Officer",
        "Sustainability Analyst",
        "Technology Director",
        "Web Editor",
        "World Designer",
    ],
    "Marketing Communications Specialist": [
        "Account Director",
        "Account Executive",
        "Administrator",
        "Assistant Dean",
        "Assistant Principal",
        "Attache",
        "Bank President",
        "Building Principal",
        "Business Executive",
        "Business Planner",
        "Chairperson",
        "Chief Administrative Officer",
        "Chief Executive Officer (CEO)",
        "Chief Learning Officer",
        "City Manager",
        "Clothier",
        "College Administrator",
        "College of Education Dean",
        "Commandant",
        "Corporate Executive",
        "Corporate Manager",
        "Corporate Officer",
        "Corporate Trainer",
        "Dean",
        "Department Head",
        "Deputy Commissioner",
        "Designer",
        "Development Vice President",
        "Director of Operations",
        "Director of Professional Services",
        "Director of Public Relations",
        "Director of Training",
        "Diversity Manager",
        "Executive Coach",
        "External Relations Director",
        "Field Manager",
        "Field Organizer",
        "Finance Vice President",
        "Fire Assistant",
        "Foundation Director",
        "Fund Raiser",
        "Fundraising Consultant",
        "Fundraising Director",
        "House Principal",
        "Human Resources Director",
        "Human Resources Manager",
        "Kitchen Manager",
        "Learning Administrator",
        "Market Manager",
        "Marketing Associate",
        "Marketing Communications Specialist",
        "Media Executive",
        "Membership Director",
        "Nonprofit Fundraiser",
        "Nonprofit Manager",
        "Office Manager",
        "Operations Officer",
        "Operations Vice President",
        "Program Coordinator",
        "Public Relations Director",
        "Relations Specialist",
        "School Coordinator",
        "School Supervisor",
        "Secondary School Principal",
        "Social Media Strategist",
        "Social Services Manager",
        "Staff Assistant",
        "Strategic Planning Director",
        "Supervisor",
        "Training Consultant",
        "Training Coordinator",
        "Training Development Director",
        "Training Director",
        "Training Facilitator",
    ],
    "Network Manager": [
        "Aeronautical Engineer",
        "Aircraft Engineer",
        "Art Director",
        "Aviation Engineer",
        "Bookmaker",
        "Business Systems Analyst",
        "Cattle Farmer",
        "Channel Manager",
        "Chartered Accountant",
        "Chief Engineer",
        "City Collector",
        "Communications Manager",
        "Compliance Analyst",
        "Computer Graphic Designer",
        "Computer Systems Engineer",
        "Computer Trainer",
        "Content Manager",
        "Contract Manager",
        "Cropper",
        "Data Coordinator",
        "Delivery Manager",
        "Development Director",
        "Development Manager",
        "Dispatcher",
        "Exhibitor",
        "Facilities Manager",
        "Facility Manager",
        "Farm Manager",
        "Finance Manager",
        "Gaming Manager",
        "Golf Course Manager",
        "Headmaster",
        "Horse Racing Manager",
        "Horse Trader",
        "reader Exporter",
        "IT Manager",
        "IT Project Manager",
        "Jobber",
        "Laboratory Scientist",
        "Logistician",
        "Manager of IT",
        "Medical Manager",
        "Network Designer",
        "Network Manager",
        "Night Manager",
        "Personal Banker",
        "Plant Manager",
        "Poker Manager",
        "Process Engineer",
        "Professor",
        "Program Manager",
        "Project Administrator",
        "Project Coordinator",
        "Project Engineer",
        "Purchaser",
        "Purveyor",
        "Quantity Surveyor",
        "Racing Manager",
        "Safety Manager",
        "Senior Group Manager",
        "Sports Team Manager",
        "Technical Specialist",
        "Telecom Engineer",
        "Telecommunications Engineer",
        "Traffic Manager",
        "Train Master",
        "Voice Engineer",
        "Warehouse Manager",
    ],
    "Recruiting Manager": [
        "Accounting Consultant",
        "Advertising Executive",
        "Business Development Executive",
        "Business Development Officer",
        "Business Development Specialist",
        "Business Services Director",
        "Career Development Director",
        "Career Specialist",
        "Commercial Banker",
        "Compliance Director",
        "Corporate Recruiter",
        "Credit Specialist",
        "Digital Marketing Strategist",
        "Director of Compliance",
        "Email Marketing Specialist",
        "Employment Manager",
        "Executive Recruiter",
        "Financial Advisor",
        "Financial Coach",
        "Financial Professional",
        "Head of Advertising",
        "Headhunter",
        "Internet Marketer",
        "Job Recruiter",
        "Marketer",
        "Marketing Agent",
        "Marketing Consultant",
        "Marketing Executive",
        "Marketing Officer",
        "Marketing Researcher",
        "Marketing Strategist",
        "Media Marketing Director",
        "Network Analyst",
        "Online Marketing Manager",
        "Principal Strategist",
        "Program Strategist",
        "Property Manager",
        "Real Estate Developer",
        "Real Estate Investor",
        "Real Estate Manager",
        "Recruiter",
        "Recruiting Manager",
        "Recruiting Specialist",
        "Regional Sales Director",
        "Resume Writer",
        "Revenue Director",
        "Revenue Officer",
        "Sales and Marketing Director",
        "Sales Officer",
        "Sales Recruiter",
        "Sales Representative",
        "Sales Trainer",
        "Sales Vice President",
        "Scout Executive",
        "Sourcing Manager",
        "Sourcing Specialist",
        "Staffing Manager",
        "Strategic Marketing Manager",
        "Talent Acquisition Manager",
        "Talent Acquisition Specialist",
        "Talent Recruiter",
        "Talent Scout",
        "Top Executive",
        "Training Executive",
        "Vice President of Marketing",
        "VP Marketing",
    ],
    "Service Advisor": [
        "Accountant",
        "Accounting Manager",
        "Advertising Analyst",
        "Appraiser",
        "Audit Manager",
        "Bank Advisor",
        "Bank Consultant",
        "Bank Officer",
        "Building Consultant",
        "Business Banker",
        "Business Consultant",
        "Business Developer",
        "Business Intelligence Manager",
        "Business Management Consultant",
        "Chief Innovation Officer",
        "Chief Pilot",
        "City Administrator",
        "Commercial Specialist",
        "Communications Analyst",
        "Compliance Manager",
        "Compliance Specialist",
        "Conference Producer",
        "Construction Consultant",
        "Construction Contractor",
        "Construction Director",
        "Contract Specialist",
        "Credit Manager",
        "Customer Service Manager",
        "Division Head",
        "Educational Administrator",
        "Employment Consultant",
        "Estate Planner",
        "Exporter",
        "Field Agent",
        "Finance Consultant",
        "Finance Officer",
        "Finance Specialist",
        "Financial Accountant",
        "Financial Consultant",
        "Financial Officer",
        "Financial Planner",
        "Financial Services Director",
        "Financier",
        "General Contractor",
        "Generation Manager",
        "Green Marketer",
        "Healthcare Recruiter",
        "Implementation Consultant",
        "Implementation Specialist",
        "Information Technology Consultant",
        "Information Technology Manager",
        "Information Technology Specialist",
        "Instructor",
        "Internet Marketing Manager",
        "Investment Advisor",
        "Investment Consultant",
        "Landman",
        "Logistics Director",
        "Logistics Specialist",
        "Maintenance Specialist",
        "Management Consultant",
        "Management Development Specialist",
        "Manufacturing Director",
        "Manufacturing Manager",
        "Market Master",
        "Marketing Administrator",
        "Marketing Analyst",
        "Networking Specialist",
        "New Media Strategist",
        "Oil Producer",
        "Online Facilitator",
        "Opener",
        "Operations Specialist",
        "Planner",
        "Procurement Specialist",
        "Project Specialist",
        "Real Estate Analyst",
        "Real Estate Officer",
        "Renewable Energy Consultant",
        "Risk Management Specialist",
        "Safety Instructor",
        "Safety Specialist",
        "Sales Manager",
        "Senior Planner",
        "Service Advisor",
        "Supply Chain Manager",
        "Support Specialist",
        "Sustainability Consultant",
        "Sustainability Specialist",
        "Tax Advisor",
        "Tax Consultant",
        "Tax Manager",
        "Technology Coordinator",
        "Tradesman",
        "Training Specialist",
        "Transportation Director",
        "Transportation Manager",
        "Transportation Specialist",
        "Union Representative",
        "Web Site Designer",
        "Website Designer",
        "Website Developer",
    ],
    "Software Programmer": [
        ".NET Developer",
        ".NET Web Developer",
        "Android Developer",
        "Applied Mathematician",
        "Beef Farmer",
        "Bioinformatician",
        "Cartographer",
        "Compiler",
        "Computational Linguist",
        "Computational Scientist",
        "Computer Engineer",
        "Computer Programmer",
        "Computer Scientist",
        "Computer Software Engineer",
        "Cryptographer",
        "Data Analyst",
        "Data Engineer",
        "Designer/Writer",
        "DevOps Engineer",
        "Director of Engineering",
        "Engineering Manager",
        "Engineering Programmer",
        "Front End Developer",
        "Game Designer",
        "Game Developer",
        "Game Engineer",
        "Game Programmer",
        "Game Tester",
        "Gaming Director",
        "Geologist",
        "Grain Farmer",
        "Infrastructure Engineer",
        "Interface Designer",
        "iOS Developer",
        "Java Developer",
        "Java Programmer",
        "Lead Designer",
        "Lead Game Designer",
        "Linux Engineer",
        "Map Maker",
        "Mathematician",
        "Network Engineer",
        "Principal Architect",
        "Principal Engineer",
        "Product Manager",
        "Programmer",
        "Programmer Analyst",
        "QA Manager",
        "Quality Analyst",
        "Release Manager",
        "Research Scientist",
        "Reviewer",
        "Security Engineer",
        "Senior Game Designer",
        "Senior Software Engineer",
        "Server Engineer",
        "Site Manager",
        "Software Architect",
        "Software Consultant",
        "Software Designer",
        "Software Developer",
        "Software Engineer",
        "Software Programmer",
        "Software Quality Engineer",
        "Software Systems Engineer",
        "Software Tester",
        "Software Writer",
        "SQL Developer",
        "System Programmer",
        "Systems Administrator",
        "Systems Analyst",
        "Systems Developer",
        "Systems Engineer",
        "Systems Programmer",
        "Systems Software Engineer",
        "Technical Architect",
        "Technical Director",
        "Test Engineer",
        "Video Game Creator",
        "Video Game Designer",
        "Video Game Developer",
        "Video Game Engineer",
        "Video Game Maker",
        "Video Game Programmer",
        "Videogame Designer",
        "Web Developer",
        "Web Engineer",
        "Web Programmer",
        "Web Software Engineer",
    ],
    "Student Services Director": [
        "Academic Coordinator",
        "Academic Vice President",
        "Admissions Director",
        "Assistant Superintendent",
        "Bridal Consultant",
        "Campus Director",
        "City Superintendent",
        "College President",
        "Community Fundraiser",
        "Community Relations Director",
        "Consul",
        "Counseling Director",
        "Counselor",
        "County Superintendent of Schools",
        "Dean of Students",
        "Director of Admissions",
        "Director of Athletics",
        "Director of Fundraising",
        "Education Supervisor",
        "Elementary Assistant Principal",
        "Elementary Principal",
        "Elementary School Principal",
        "Executive Administrator",
        "Fundraiser",
        "Fundraising Manager",
        "High School Principal",
        "Mayor",
        "Middle School Principal",
        "Principal",
        "School Administrator",
        "School Principal",
        "School Superintendent",
        "Site Coordinator",
        "Student Affairs Dean",
        "Student Affairs Vice President",
        "Student Dean",
        "Student Services Director",
        "Superintendent",
        "Superintendent of Schools",
        "University President",
        "Wedding Designer",
        "Wedding Planner",
        "Wellness Ambassador",
        "Wellness Coach",
        "Wellness Director",
        "Wellness Manager",
    ],
    "U.S. Representative": [
        "Congressional Representative",
        "Congressman",
        "County Commissioner",
        "County Supervisor",
        "County Treasurer",
        "Delegate",
        "District Leader",
        "Governor",
        "Inspector",
        "Legislator",
        "Lieutenant Governor",
        "Member of Congress",
        "Policy Officer",
        "Representative",
        "Secretary of State",
        "Senator",
        "State Auditor",
        "U.S. Representative",
        "U.S. Senator",
    ],
    "Web Specialist": [
        "Analytics Consultant",
        "Business Intelligence Analyst",
        "Business Process Consultant",
        "Collector",
        "Computer Specialist",
        "Controller",
        "Course Developer",
        "Cyber Security Analyst",
        "Dairyman",
        "Database Architect",
        "Director of Information",
        "Energy Analyst",
        "Energy Consultant",
        "Estimator",
        "Forensic Accountant",
        "Grower",
        "Information Director",
        "Information Technology Director",
        "Intelligence Analyst",
        "Internet Project Manager",
        "Internet Researcher",
        "Internet Specialist",
        "Internet Webmaster",
        "Manager",
        "Pig Farmer",
        "Planning Analyst",
        "Policy Analyst",
        "Process Analyst",
        "Registrar",
        "Regulatory Consultant",
        "Research Analyst",
        "Security Analyst",
        "Security Director",
        "Security Manager",
        "Security Specialist",
        "Senior Program Manager",
        "Software Publisher",
        "Software Trainer",
        "Storage Manager",
        "Surveyor",
        "Systems Consultant",
        "Systems Development Manager",
        "Tax Specialist",
        "Teacher",
        "Technical Project Manager",
        "Technology Consultant",
        "Technology Project Manager",
        "User Experience Designer (UX Designer)",
        "Web Analyst",
        "Web Content Developer",
        "Web Designer",
        "Web Graphic Designer",
        "Web Specialist",
        "Webmaster",
    ],
}

time: 152 ms (started: 2023-06-26 22:02:25 -07:00)


In [5]:
cluster_colors = [
    "#FB9A99",
    "#E31A1C",
    "#33A02C",
    "#B15928",
    "#CAB2D6",
    "#E7298A",
    "#A6CEE3",
    "#6A3D9A",
    "#A6761D",
    "#666666",
    "#1B9E77",
    "#66A61E",
    "#E6AB02",
    "#FDBF6F",
    "#B2DF8A",
    "#1F78B4",
    "#D95F02",
    "#7570B3",
    "#7FC97F",
    "#FF7F00",
]

time: 3.19 ms (started: 2023-06-26 22:02:25 -07:00)


All that's left is to do some inverting of indices and mapping to colors and storing in a dataframe, and also reindexing so we have the same order as the `data`:

In [6]:
professions_list = []
cluster_list = []

for cluster_name, professions in clusters.items():
    professions_list.extend(professions)
    cluster_list.extend([cluster_name] * len(professions))

target = pd.DataFrame({"Cluster_Name": cluster_list}).set_index(
    pd.Index(professions_list)
)
target = target.reindex(data.index)
target

,Cluster_Name
Profession,
.Net Architect,Information Engineer
.NET Developer,Software Programmer
.NET Programmer,Information Engineer
.NET Web Developer,Software Programmer
Academic Coordinator,Student Services Director
...,...
Wine Maker,Communications Specialist
Wireless Engineer,Information Engineer
World Designer,Management Scientist


time: 14 ms (started: 2023-06-26 22:02:25 -07:00)


In [7]:
target_palette = dict(Cluster_Name=dict(zip(clusters.keys(), cluster_colors)))
target_palette

{'Cluster_Name': {'Business Analyst': '#FB9A99',
  'Certified Financial Planner (CFP)': '#E31A1C',
  'City Alderman': '#33A02C',
  'Communications Specialist': '#B15928',
  'Concert Manager': '#CAB2D6',
  'Director of Education': '#E7298A',
  'District Manager': '#A6CEE3',
  'Fashion Marketer': '#6A3D9A',
  'Group Manager': '#A6761D',
  'Health Care Manager': '#666666',
  'Information Engineer': '#1B9E77',
  'Management Scientist': '#66A61E',
  'Marketing Communications Specialist': '#E6AB02',
  'Network Manager': '#FDBF6F',
  'Recruiting Manager': '#B2DF8A',
  'Service Advisor': '#1F78B4',
  'Software Programmer': '#D95F02',
  'Student Services Director': '#7570B3',
  'U.S. Representative': '#7FC97F',
  'Web Specialist': '#FF7F00'}}

time: 5.46 ms (started: 2023-06-26 22:02:25 -07:00)


In [8]:
from drnb.io.pipeline import create_default_pipeline

data_pipe = create_default_pipeline(
    check_for_duplicates=True,
)

data_result = data_pipe.run(
    "vocation-map",
    data=data,
    target=target,
    target_palette=target_palette,
    url="https://doi.org/10.1073/pnas.1917942116",
    tags=["small", "lowdim"],
    verbose=True,
)

[22:02:30] INFO     Requesting one extra neighbor to account for self-neighbor                      ]8;id=21676;file:///home/james/dev/drnb/src/drnb/neighbors/__init__.py\__init__.py]8;;\:]8;id=42455;file:///home/james/dev/drnb/src/drnb/neighbors/__init__.py#569\569]8;;\

           INFO     Initial data shape: (1227, 10)                                                   ]8;id=511489;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=8808;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#75\75]8;;\

           INFO     Keeping all columns                                                            ]8;id=322266;file:///home/james/dev/drnb/src/drnb/preprocess.py\preprocess.py]8;;\:]8;id=574168;file:///home/james/dev/drnb/src/drnb/preprocess.py#72\72]8;;\

           INFO     Data shape after filtering columns: (1227, 10)                                  ]8;id=75378;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=816910;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#146\146]8;;\

           INFO     Removing rows with NAs                                                          ]8;id=904622;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=64958;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#130\130]8;;\

...

           INFO     Writing csv format to triplets/vocation-map.5.1337.euclidean.csv                ]8;id=12113;file:///home/james/dev/drnb/src/drnb/io/__init__.py\__init__.py]8;;\:]8;id=684236;file:///home/james/dev/drnb/src/drnb/io/__init__.py#269\269]8;;\

           INFO     Writing numpy format to triplets/vocation-map.5.1337.idx.npy                    ]8;id=74859;file:///home/james/dev/drnb/src/drnb/io/__init__.py\__init__.py]8;;\:]8;id=803382;file:///home/james/dev/drnb/src/drnb/io/__init__.py#292\292]8;;\

           INFO     Writing numpy format to triplets/vocation-map.5.1337.euclidean.npy              ]8;id=28111;file:///home/james/dev/drnb/src/drnb/io/__init__.py\__init__.py]8;;\:]8;id=297330;file:///home/james/dev/drnb/src/drnb/io/__init__.py#292\292]8;;\

[22:02:34] INFO     Writing pipeline result for vocation-map                                        ]8;id=109987;file:///home/james/dev/drnb/src/drnb/io/pipeline.py\pipeline.py]8;;\:]8;id=394137;file:///home/james/dev/drnb/src/drnb/io/pipeline.py#123\123]8;;\

time: 8.19 s (started: 2023-06-26 22:02:25 -07:00)
